# Desafio - Aula 2

In [69]:
import pandas as pd
from unidecode import unidecode
import re
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import spacy
from collections import Counter
from nltk.stem.rslp import RSLPStemmer
nltk.download('rslp')
nltk.download('punkt')
nlp = spacy.load('pt_core_news_sm')
nltk.download('stopwords')

df = pd.read_csv("https://s3.amazonaws.com/automl-example/produtos.csv",
                 delimiter=";", encoding='utf-8')

[nltk_data] Downloading package rslp to /home/guilherme/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /home/guilherme/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/guilherme/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1) utilizando o df acima carregado, faça:

   ### -> Elimine linhas com valores nulos
   ### -> Adicione uma nova coluna chamada texto, formada pela composição das colunas nome e descrição
   ### -> Quantos Unigramas existem antes e depois de remover stopwords
   ### -> Quantos Bigramas existem antes e depois de remover stopwords
   ### -> Quantos Trigramas existem antes e depois de remover stopwords
   ### -> Quantos verbos e Adverbios existem na nova coluna (utilize: from collections import Counter)
   ### -> Quantos unigramas existem após aplicar Stemmer (utilize rslp)

In [70]:
df.dropna(inplace=True)
df.isna().any()

nome         False
descricao    False
categoria    False
dtype: bool

In [71]:
df['texto'] = df['nome'] + ' ' + df['descricao']

In [72]:
stops = nltk.corpus.stopwords.words('portuguese')

In [87]:
def clearText(text):    
    ## Pontuação
    text = [re.sub('[^A-Za-z ]+', '', word) for word in text]
    ## Retirar numeros
    text = [re.sub('[0-9]+', '', word) for word in text]
    ## Retirar caracteres especiais e letras soltas
    text = [unidecode(name) for name in text if name and len(name) > 1]
    ## minuscolas
    text = [name.lower() for name in text]
        
#     text = [word_tokenize(line.lower()) for line in text] # caixa baixa
#     text = [[w for w in line if w not in stops]for line in text]# remove stop words
#     text = [' '.join(line) for line in text]# remove stop words
    
    return text

In [74]:
def getNgram(words, n):
    
    vect = CountVectorizer(ngram_range=(n, n))
    vect.fit(words)
    text_vect = vect.transform(words)

    print('Unigramas com stop words: ' + str(len(vect.get_feature_names())))
    # pd.DataFrame(text_vect.A, columns=names)

    ##########################################################################
    
    vect = CountVectorizer(ngram_range=(n, n), stop_words=stops)
    vect.fit(words)
    text_vect = vect.transform(words)

    print('Unigramas sem stop words: ' + str(len(vect.get_feature_names())))
    
    # pd.DataFrame(text_vect.A, columns=names)

    return vect.get_feature_names()


In [75]:
words = df.texto
words = clearText(words)

ngram = getNgram(words, 1)

Unigramas com stop words: 35604
Unigramas sem stop words: 35465


In [51]:
ngram = getNgram(words, 2)

Unigramas com stop words: 149715
Unigramas sem stop words: 136933


In [52]:
ngram = getNgram(words, 3)

Unigramas com stop words: 212541
Unigramas sem stop words: 165079


### Separando as palavras unicas e aplicando Stemmer

In [98]:
words = df.texto
words = clearText(words)
words = ' '.join(words)
words = words.split(' ')
words = [name for name in words if name]
print(len(words))

print(len(set(words)))
# print(set(words))
rslp = RSLPStemmer()
stm = [rslp.stem(word) for word in set(words) if len(word)>1]

print(len(stm))

518684
35630
35604


In [101]:
words = df.texto
words = clearText(words)
words = ' '.join(words)
words = word_tokenize(words.lower())
words = [w for w in words if w not in stops]
words = [' '.join(line) for line in words]# join words
    
##############Estoura o limite de caracteres da lib spacy
doc = nlp(' '.join(words))
c = Counter([token.pos_ for token in doc])
###############

# teste = [nlp(unicode(w) for w in words]
# c = [Counter([token.pos_ for token in word]) for word in teste]
c
# print(c['ADV'])
# print(c['VERB'])

TypeError: Argument 'string' has incorrect type (expected unicode, got list)

In [ ]:
words

### 2) crie um tagger baseado em expressões regulares:
   ### -> crie uma variável que amazenará uma tupla (expressão, tag)
   ### -> o tagger deverá capturar gerúndio (ando, endo, indo), plurais e números cardinais
   ### -> utilize nltk.RegexpTagger(variável) para carregar seu tagger
   ### -> apresente uma frase teste para cada tipo de expressão

In [ ]:
import re

tags = [
    (r'.+ando|endo|indo&', 'gerundio'),
    (r'.+s$', 'plural'),
    (r'[0-9]+', 'numeros')
]

In [ ]:
import nltk
tagger = nltk.RegexpTagger(tags)

In [ ]:
text = "Veja os 2 animais andando na rua"
tolk = nltk.word_tokenize(text)
tagger.tag(tolk)